# Imports

In [1]:
import torch
import sys
import numpy as np
import os
import shutil
from pathlib import Path
# Add parent directory to sys.path
parent_dir = Path.cwd().parent.parent
sys.path.append(str(parent_dir))
# Import modules
from train import train_test_multiple_networks, train_network_with_test_loss, train_grad_network_with_test_loss, save_plot_test_train

# Network Parameters

In [2]:
# Training hyperparameters
num_trials = 10
neurons = [64, 64, 64]
num_epochs = 200

# Running Experiments

In [3]:
# Folder containing datasets
dataset_folder = "datasets"

# Output folder for plots
output_folder = "loss_plots"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

for subfolder in os.listdir(dataset_folder):
    subfolder_path = os.path.join(dataset_folder, subfolder)
    
    if os.path.isdir(subfolder_path):
        print(f"Processing folder: {subfolder}")

        output_subfolder = os.path.join(output_folder, subfolder)
        os.makedirs(output_subfolder, exist_ok=True)
        
        train_folder = os.path.join(subfolder_path, "train")
        test_folder = os.path.join(subfolder_path, "test")
        
        for filename in os.listdir(train_folder):
            if filename.endswith("_train.txt"):
                base_name = filename.replace("_train.txt", "")
                
                # Load train data
                train_filepath = os.path.join(train_folder, filename)
                train_data = np.loadtxt(train_filepath)

                dimension = (int)(train_data.shape[1]/2)

                train_points = torch.tensor(train_data[:, :dimension], dtype=torch.float32)
                train_values = torch.tensor(train_data[:, dimension:], dtype=torch.float32)

                
                # Load test data
                test_filename = f"{base_name}_test.txt"
                test_filepath = os.path.join(test_folder, test_filename)
                test_data = np.loadtxt(test_filepath)
                test_points = torch.tensor(test_data[:, :dimension], dtype=torch.float32)
                test_values = torch.tensor(test_data[:, dimension:], dtype=torch.float32)
                
                print(f"Loaded preprocessed data for dimension {dimension}")
                
                # Train the normal network
                mean_loss, variance_loss, mean_test_loss, variance_test_loss = train_test_multiple_networks(
                    train_network_with_test_loss, dimension, train_points, neurons, 
                    num_epochs, train_values, test_points, test_values, num_trials)
                print("Done training the normal network!")
        
                # Train the conservative network
                mean_loss_grad, variance_loss_grad, mean_test_loss_grad, variance_test_loss_grad = train_test_multiple_networks(
                    train_grad_network_with_test_loss, dimension, train_points, neurons, 
                    num_epochs, train_values, test_points, test_values, num_trials)
                print("Done training the conservative network!")

                # Calculate standard deviations for the normal network
                std_loss = np.sqrt(variance_loss)
                std_test_loss = np.sqrt(variance_test_loss)
                std_loss_grad = np.sqrt(variance_loss_grad)
                std_test_loss_grad = np.sqrt(variance_test_loss_grad)
        
                # Print the minimum loss
                print(f"The minimum loss was {mean_loss[-1]}, conservative {mean_loss_grad[-1]}")
                print(f"The minimum test loss was {mean_test_loss[-1]}, conservative {mean_test_loss_grad[-1]}")
                # Comparison Plot
                epochs = np.arange(num_epochs)
                save_plot_test_train(epochs, mean_loss, std_loss, mean_loss_grad, std_loss_grad,
                      mean_test_loss, std_test_loss, mean_test_loss_grad, std_test_loss_grad,
                      dimension, neurons, num_epochs, output_subfolder)

                # Save the losses to .txt files
                losses_filename = os.path.join(output_subfolder, f"{dimension}D_losses.txt")
                np.savetxt(
                    losses_filename,
                    np.column_stack((
                        epochs,
                        mean_loss, std_loss,  # Training loss and std for normal network
                        mean_test_loss, std_test_loss,  # Test loss and std for normal network
                        mean_loss_grad, std_loss_grad,  # Training loss and std for conservative network
                        mean_test_loss_grad, std_test_loss_grad  # Test loss and std for conservative network
                    )),
                    fmt='%.6f',
                    header='epochs mean_loss std_loss mean_test_loss std_test_loss mean_loss_grad std_loss_grad mean_test_loss_grad std_test_loss_grad'
                )
                print(f"Losses saved to {losses_filename}")
        
print ("Done!!!")

Using device: cuda
Processing folder: dataset_1
Loaded preprocessed data for dimension 10
The network training took 6.45 seconds.
Done training the normal network!
The network training took 7.60 seconds.
Done training the conservative network!
The minimum loss was 4.1423206494073386e-05, conservative 8.353283737960737e-06
The minimum test loss was 4.5803111061104576e-05, conservative 8.32079258543672e-06
Plot saved to loss_plots\dataset_1\10D_Conservative_NN.png
Losses saved to loss_plots\dataset_1\10D_losses.txt
Loaded preprocessed data for dimension 12
The network training took 3.80 seconds.
Done training the normal network!
The network training took 8.04 seconds.
Done training the conservative network!
The minimum loss was 2.2299320335150696e-05, conservative 3.933459652216698e-06
The minimum test loss was 2.5975828430091495e-05, conservative 4.2083808466486515e-06
Plot saved to loss_plots\dataset_1\12D_Conservative_NN.png
Losses saved to loss_plots\dataset_1\12D_losses.txt
Loaded p

# Organizing Data

In [4]:
# Folder containing the loss plots and text files
loss_folder = "loss_plots"

# Output folders for consolidated images and text files
output_image_folder = "consolidated_images"
output_text_folder = "consolidated_text_files"

# Create output folders if they don't exist
os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_text_folder, exist_ok=True)

# Iterate through all subfolders in the loss_folder
for subfolder in os.listdir(loss_folder):
    subfolder_path = os.path.join(loss_folder, subfolder)
    
    # Check if it's a directory (e.g., dataset_1, dataset_2, etc.)
    if os.path.isdir(subfolder_path):
        print(f"Processing folder: {subfolder}")
        
        # Extract the dataset number (e.g., "1" from "dataset_1")
        dataset_number = subfolder.split("_")[-1]
        
        # Process images
        for filename in os.listdir(subfolder_path):
            if filename.endswith(".png"):  # Process only .png files
                # Construct the new filename (e.g., 2D_Conservative_NN_1.png)
                new_filename = filename.replace(".png", f"_{dataset_number}.png")
                
                # Copy the image to the consolidated folder
                src_path = os.path.join(subfolder_path, filename)
                dest_path = os.path.join(output_image_folder, new_filename)
                shutil.copy(src_path, dest_path)
                #print(f"Copied image: {src_path} -> {dest_path}")
        
        # Process text files
        for filename in os.listdir(subfolder_path):
            if filename.endswith(".txt"):  # Process only .txt files
                # Construct the new filename (e.g., 8D_losses_1.txt)
                new_filename = filename.replace(".txt", f"_{dataset_number}.txt")
                
                # Copy the text file to the consolidated folder
                src_path = os.path.join(subfolder_path, filename)
                dest_path = os.path.join(output_text_folder, new_filename)
                shutil.copy(src_path, dest_path)
                #print(f"Copied text file: {src_path} -> {dest_path}")

print("All files have been consolidated.")

Processing folder: dataset_1
Processing folder: dataset_10
Processing folder: dataset_2
Processing folder: dataset_3
Processing folder: dataset_4
Processing folder: dataset_5
Processing folder: dataset_6
Processing folder: dataset_7
Processing folder: dataset_8
Processing folder: dataset_9
All files have been consolidated.
